# DEFCoN package

A quick workflow guide to DEFCoN (Density Estimation Fully Convolutional Network)

## Data collection
### TrainingSet
A training set for DEFCoN is made of input SMLM images, with their segmentation masks and density maps built from the ground truth of the fluorophores. The training images are built using SASS. The simulation must output the generated tiff stack as well as a *frame logger* csv file (FrameLogger class in SASS). The TrainingSet class is used to store the data. It is inherited from an h5py File.

The following creates a TrainingSet:

In [1]:
from DEFCoN.datasets import TrainingSet

h5_path = 'example/training/my_training_set.h5'
# Create h5 file, replace if exists
my_training_set = TrainingSet(h5_path, 'w')

At this point, the TrainingSet is empty

In [2]:
my_training_set.summary()


example/training/my_training_set.h5
/
    input/
    target/



The methods add_dataset is used to add the input images to the TrainingSet, and build their density maps and segmentation masks from the ground truth.

In [3]:
from IPython.display import clear_output

dataset_name = 'first_dataset'
my_training_set.add_dataset(name=dataset_name,
                            input_file='example/data_1/generated_stack.tif',
                            frame_logger='example/data_1/frame_logger.csv')

clear_output()

The TrainingSet now looks like this:

In [4]:
my_training_set.summary()


example/training/my_training_set.h5
/
    input/
        first_dataset: shape = (1500, 64, 64, 1), dtype = float64
    seg_map/
        first_dataset: shape = (1500, 64, 64, 1), dtype = int32
    target/
        first_dataset: shape = (1500, 64, 64, 1), dtype = float64



Adding a second dataset, the TrainingSet becomes:

In [5]:
dataset_name = 'second_dataset'
my_training_set.add_dataset(name=dataset_name,
                            input_file='example/data_2/generated_stack.tif',
                            frame_logger='example/data_2/frame_logger.csv')

clear_output()
my_training_set.summary()


example/training/my_training_set.h5
/
    input/
        first_dataset: shape = (1500, 64, 64, 1), dtype = float64
        second_dataset: shape = (1500, 64, 64, 1), dtype = float64
    seg_map/
        first_dataset: shape = (1500, 64, 64, 1), dtype = int32
        second_dataset: shape = (1500, 64, 64, 1), dtype = int32
    target/
        first_dataset: shape = (1500, 64, 64, 1), dtype = float64
        second_dataset: shape = (1500, 64, 64, 1), dtype = float64



Always close the TrainingSet when you're done using it. It's a limitation of the HDF5 format.

In [6]:
my_training_set.close()

### Compact TrainingSet
To train faster, a TrainingSet containing multiple datasets can be compacted in a single large dataset, provided that _all input images have the same dimensions_.The images can be augmented (by random brightness changes) and shuffled in the process

In [7]:
from DEFCoN.datasets import compact_set

# This time, we open my_training_set in read_only mode
my_training_set = TrainingSet('example/training/my_training_set.h5', 'r')

# Output a compact training set named 'my_compact_set.h5'
compact_set(my_training_set,
            output_file='example/training/my_compact_set.h5',
            shuffle=True,
            augment=True)

# Close my_training_set
my_training_set.close()

Compacting input data...
    first_dataset...
    second_dataset...
Augmenting data...
Compacting seg_map data...
    first_dataset...
    second_dataset...
Compacting target data...
    first_dataset...
    second_dataset...


The result:

In [8]:
my_compact_set = TrainingSet('example/training/my_compact_set.h5')
my_compact_set.summary()
my_compact_set.close()


example/training/my_compact_set.h5
/
    input/
        input: shape = (3000, 64, 64, 1), dtype = float64
    seg_map/
        seg_map: shape = (3000, 64, 64, 1), dtype = int32
    target/
        target: shape = (3000, 64, 64, 1), dtype = float64



## Training
The best method for training DEFCoN is by traning first the segmentation network, then the density network. The current architecture can be found in DEFCoN.models. The package uses the FCN class to add methods to the Keras Model class. The most efficient method for training is to train on a compact set of augmented and shuffled images. It is implemented in the "train" method from the DEFCoN.training module. The parameters are input using a config ini file. Taking a look at our example config file:

The first section just deals with the paths of the TrainingSet and the output model and weights. Note that the paths to the directories and training set file are **relative to the config file**. The other sections let the user tune the parameters of the training. To train a model, simply run:

In [ ]:
from DEFCoN import training

training.train('example/training/config.ini')

The training script can also be run from the command line

## Prediction
Predictions can be made on arrays (TrainingSet format) or on images. DEFCoN.networks.FCN is a class inheriting methods from the Keras Model class and with added methods for prediction and serving.

In [9]:
from DEFCoN.networks import FCN
from DEFCoN.datasets import TrainingSet
import numpy as np

# Import the trained model from the saved h5 file
model = FCN.from_file('example/training/trained_models/my_trained_model.h5')

# Predict from array
my_training_set = TrainingSet('example/training/my_training_set.h5', 'r')
array = np.array(my_training_set['input/first_dataset'])
my_training_set.close()

y_pred = model.predict(array)

# Predict from image stack
y_pred = model.predict_tiff('example/data_1/generated_stack.tif')

Using TensorFlow backend.


The prediction is a stack of density map, as a numpy array with 4 dimensions: (N_frames, x, y, 1). The last dimension is the number of channels (1 because it is grayscale).

In [10]:
y_pred.shape

(1500, 64, 64, 1)

## Serving for Java

Trained DEFCoN model can be exported to be used in the Java implementation of DEFCoN.

In [ ]:
from DEFCoN.networks import FCN

# Import the trained model from the saved h5 file
model = FCN.from_file('example/training/trained_models/my_trained_model.h5')

# Save it for serving
model.save_tf_model('example/serving/tf_density_count')

The resulting folder can be directly placed in the DEFCoN plugin directory in FIJI


Fiji.app/plugins/DEFCoN/tf_density_count

### Maximum local count
Trained DEFCoN model can be converted to allow for maximum count prediction by adding layers

In [11]:
from DEFCoN.networks import FCN

# Import the trained model from the saved h5 file
model = FCN.from_file('example/training/trained_models/my_trained_model.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, None, 1)     0         
_________________________________________________________________
hist_norm (Lambda)           (None, None, None, 1)     0         
_________________________________________________________________
conv_seg_1 (Conv2D)          (None, None, None, 16)    160       
_________________________________________________________________
conv_stride_1 (Conv2D)       (None, None, None, 16)    2320      
_________________________________________________________________
conv_seg_2 (Conv2D)          (None, None, None, 32)    4640      
_________________________________________________________________
conv_stride_2 (Conv2D)       (None, None, None, 32)    9248      
_________________________________________________________________
conv_seg_3 (Conv2D)          (None, None, None, 64)    18496     
__________

In [12]:
# Adding prediction layer for max local count with 7x7 windows
model.density_to_max_count(pool=(7,7))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, None, None, 1)     0         
_________________________________________________________________
hist_norm (Lambda)           (None, None, None, 1)     0         
_________________________________________________________________
conv_seg_1 (Conv2D)          (None, None, None, 16)    160       
_________________________________________________________________
conv_stride_1 (Conv2D)       (None, None, None, 16)    2320      
_________________________________________________________________
conv_seg_2 (Conv2D)          (None, None, None, 32)    4640      
_________________________________________________________________
conv_stride_2 (Conv2D)       (None, None, None, 32)    9248      
_________________________________________________________________
conv_seg_3 (Conv2D)          (None, None, None, 64)    18496     
__________

The resulting model can then be served for Java:

In [ ]:
model.save_tf_model('example/serving/tf_max_count')